In [ ]:


import os, cv2, random, glob, numpy as np, pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

BASE_DIR = '/../FIVES A Fundus Image Dataset for AI-based Vessel Segmentation'
TRAIN_DIR = os.path.join(BASE_DIR, 'train', 'Original')
TEST_DIR  = os.path.join(BASE_DIR, 'test', 'Original')
META_FILE = os.path.join(BASE_DIR, 'Quality Assessment.xlsx')

RESULTS_DIR = '/../FIVES_results/quality_aware'
os.makedirs(RESULTS_DIR, exist_ok=True)

IMG_SIZE = (224,224)
BATCH_SIZE = 8
EPOCHS = 25
AUTOTUNE = tf.data.AUTOTUNE


In [ ]:
meta = pd.read_excel(META_FILE)
print("Columns:", list(meta.columns), "Rows:", len(meta))

def find_image_path(disease, number):
    fname = f"{int(number)}_{disease}.png"
    for d in [TRAIN_DIR, TEST_DIR]:
        p = os.path.join(d, fname)
        if os.path.exists(p): return p
    return None

meta['filename'] = meta.apply(lambda r: f"{int(r['Number'])}_{r['Disease']}.png", axis=1)
meta['image_path'] = meta.apply(lambda r: find_image_path(r['Disease'], r['Number']), axis=1)
meta = meta.dropna(subset=['image_path']).reset_index(drop=True)

import json, shutil

SHARED_MAP_PATH = '/../FIVES_results/label_map.json'
if not os.path.exists(SHARED_MAP_PATH):
    raise FileNotFoundError("Shared label_map.json not found! Run the baseline notebook first.")

label_map = json.load(open(SHARED_MAP_PATH))
meta['label'] = meta['Disease'].map(label_map)
NUM_CLASSES = len(label_map)

# Copy it to this model’s result folder for consistency
shutil.copy(SHARED_MAP_PATH, os.path.join(RESULTS_DIR, 'label_map.json'))
print("✅ Loaded shared label_map:", label_map)


In [ ]:
# === Load shared splits created by the baseline model ===
SPLITS_DIR = '/../FIVES_results/splits'

train_csv = os.path.join(SPLITS_DIR, 'train.csv')
val_csv   = os.path.join(SPLITS_DIR, 'val.csv')
test_csv  = os.path.join(SPLITS_DIR, 'test.csv')

if not all(os.path.exists(p) for p in [train_csv, val_csv, test_csv]):
    raise FileNotFoundError("❌ Train/val/test split files not found. Run baseline notebook first!")

train_df = pd.read_csv(train_csv)
val_df   = pd.read_csv(val_csv)
test_df  = pd.read_csv(test_csv)

print(f"✅ Loaded shared splits → Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")



In [ ]:
def preprocess_quality(img, ic, blur, lc):
    """Apply quality-based correction."""
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)

    # Illumination correction
    if ic == 0:
        lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        l,a,b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        l = clahe.apply(l)
        img = cv2.cvtColor(cv2.merge((l,a,b)), cv2.COLOR_LAB2RGB)

    # Deblur
    if blur == 0:
        img = cv2.GaussianBlur(img, (3,3), 0)
        img = cv2.addWeighted(img, 1.5, img, -0.5, 0)

    # Contrast enhancement
    if lc == 0:
        img = cv2.convertScaleAbs(img, alpha=1.2, beta=10)

    return img

def decode_preprocess(path, ic, blur, lc, label):
    def _load_and_process(p, ic_val, bl_val, lc_val):
        p = p.decode()
        img = cv2.imread(p)
        if img is None:
            img = np.zeros((*IMG_SIZE, 3), np.uint8)
        img = preprocess_quality(img, int(ic_val), int(bl_val), int(lc_val))
        return img

    img = tf.numpy_function(_load_and_process, [path, ic, blur, lc], tf.uint8)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return img, label


def make_dataset(df, training=True):
    paths = df['image_path'].values
    labels = df['label'].values
    ic = df['IC'].values
    bl = df['Blur'].values
    lc = df['LC'].values

    ds = tf.data.Dataset.from_tensor_slices((paths, ic, bl, lc, labels))
    ds = ds.map(decode_preprocess, num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.shuffle(512, seed=SEED)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

train_ds = make_dataset(train_df)
val_ds = make_dataset(val_df, False)
test_ds = make_dataset(test_df, False)


In [ ]:
base = tf.keras.applications.EfficientNetB0(include_top=False, pooling='avg', input_shape=IMG_SIZE+(3,))
x = layers.Dropout(0.4)(base.output)
out = layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = models.Model(base.input, out)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
ckpt = os.path.join(RESULTS_DIR, 'best_quality.keras')
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(ckpt, monitor='val_accuracy', save_best_only=True, mode='max'),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, mode='max', restore_best_weights=True)
]

history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=callbacks)


In [ ]:
best_model = tf.keras.models.load_model(ckpt)
y_true, y_pred, y_prob = [], [], []

for imgs, labs in test_ds:
    probs = best_model.predict(imgs, verbose=0)
    preds = np.argmax(probs, axis=1)
    y_true.extend(labs.numpy()); y_pred.extend(preds); y_prob.extend(probs)

print(classification_report(y_true, y_pred, target_names=list(label_map.keys())))
try:
    auc = roc_auc_score(tf.keras.utils.to_categorical(y_true, NUM_CLASSES), np.array(y_prob), average='macro')
    print("Macro AUC:", round(auc,4))
except: pass

In [ ]:
import shutil
shutil.copy(SHARED_MAP_PATH, os.path.join(RESULTS_DIR, 'label_map.json'))
print("Copied shared label_map to quality_aware results folder.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc
import seaborn as sns
import tensorflow as tf



In [ ]:
import pandas as pd
# Paths
RESULTS_DIR = '/../FIVES_results/quality_aware'
MODEL_PATH = os.path.join(RESULTS_DIR, 'best_quality.keras')
BASE_DIR = '/../FIVES A Fundus Image Dataset for AI-based Vessel Segmentation'
TEST_DIR = os.path.join(BASE_DIR, 'test', 'Original')
TRAIN_DIR = os.path.join(BASE_DIR, 'train', 'Original')
META_FILE = os.path.join(BASE_DIR, 'Quality Assessment.xlsx')

# Load metadata
meta = pd.read_excel(META_FILE)

def find_image_path(disease, number):
    fname = f"{int(number)}_{disease}.png"
    for d in [TRAIN_DIR, TEST_DIR]:
        p = os.path.join(d, fname)
        if os.path.exists(p): return p
    return None

meta['image_path'] = meta.apply(lambda r: find_image_path(r['Disease'], r['Number']), axis=1)
meta = meta.dropna(subset=['image_path']).reset_index(drop=True)

# Load shared label map
import json
label_map_path = '/../FIVES_results/baseline/label_map.json'
label_map = json.load(open(label_map_path))
labels = list(label_map.keys())

# Load trained model
model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Loaded model:", MODEL_PATH)


In [ ]:
IMG_SIZE = (224,224)
BATCH_SIZE = 8
AUTOTUNE = tf.data.AUTOTUNE

def decode_resize(path, label):
    img_bytes = tf.io.read_file(path)
    img = tf.image.decode_image(img_bytes, channels=3, expand_animations=False)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32)/255.0
    return img, label

meta['label'] = meta['Disease'].map(label_map)

from sklearn.model_selection import train_test_split
_, temp_df = train_test_split(meta, test_size=0.2, stratify=meta['label'], random_state=42)
_, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

paths = test_df['image_path'].values
labels_np = test_df['label'].values
test_ds = tf.data.Dataset.from_tensor_slices((paths, labels_np))
test_ds = test_ds.map(decode_resize, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

print(f"Test dataset ready ({len(test_df)} samples)")


In [ ]:
y_true, y_pred, y_prob = [], [], []

for imgs, labs in test_ds:
    probs = best_model.predict(imgs, verbose=0)
    preds = np.argmax(probs, axis=1)
    y_true.extend(labs.numpy())
    y_pred.extend(preds)
    y_prob.extend(probs)

y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_prob = np.array(y_prob)

In [ ]:
print("\n Classification Report:")
print(classification_report(y_true, y_pred, target_names=labels))


In [ ]:
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix – Quality-Aware Model")
plt.show()


In [ ]:
plt.figure(figsize=(7,6))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

for i, label in enumerate(labels):
    fpr, tpr, _ = roc_curve(tf.keras.utils.to_categorical(y_true, len(labels))[:, i],
                            y_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=colors[i], lw=2,
             label=f'{label} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=1)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves by Class – Quality-Aware Model')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
try:
    macro_auc = roc_auc_score(tf.keras.utils.to_categorical(y_true, len(labels)),
                              y_prob, average='macro')
    print("Macro AUC:", round(macro_auc, 4))
except Exception as e:
    print("AUC computation error:", e)